# 1. Veri Seti ve Problem Tanımı

Bu sınıflandırma projesinde, hastaların kalp hastalığı olup olmadığını tahmin etmek için farklı yöntemler kullanarak yüksek doğruluklu bir makine öğrenmesi modeli oluşturulmaya çalışılmaktadır.

Kullanılacak veri seti UCI tarafından sağlanan Kalp Hastalığı veri setidir.

# 2. Özellikler

1. age
2. sex: (1 = male; 0 = female)
3. cp: (göğüs ağrısı tipi)
 * 0:Tipik anjina: göğüs ağrısına bağlı kalbe kan akışının azalması
 * 1:Atipik anjina: kalple ilgili olmayan göğüs ağrısı
 * 2:Anjina dışı ağrı: tipik olarak özofagus spazmları (kalple ilgili olmayan)
 * 3:Asemptomatik: hastalık belirtisi göstermeyen göğüs ağrısı
4. tretbps: dinlenme kan basıncı (hastaneye kabulde mm Hg cinsinden) 130-140'ın üzerinde herhangi bir şey genellikle endişe kaynağıdır.
5. chol: mg/dl cinsinden serum kolesterolü
 * serum = LDL + HDL + 0,2 * trigliseritler
 * 200'ün üstü endişe kaynağı
6. fbs: (açlık kan şekeri > 120 mg/dl) (1 = doğru; 0 = yanlış)
 * '>126' mg/dL diyabet sinyalidir
7. restecg: dinlenme elektrokardiyografik sonuçları
 * 0: Not edilecek bir şey yok
 * 1: ST-T Dalga anormalliği
   -Hafif semptomlardan ciddi sorunlara kadar değişebilir
   -normal olmayan kalp atışının sinyalini verir
 * 2: Olası veya kesin sol ventriküler hipertrofi
   -Genişlemiş kalbin ana pompalama odası
8. thalach: ulaşılan maksimum kalp atış hızı
9. exang: egzersize bağlı anjina (1 = evet; 0 = hayır)
10. oldpeak: Dinlenmeyle karşılaştırıldığında egzersizin neden olduğu ST depresyonu, egzersiz sırasında kalpteki strese bakar, sağlıksız kalp daha fazla strese neden olur
11. slope: zirve egzersiz ST segmentinin eğimi
  * 0: Yükseliş: egzersizle daha iyi kalp atış hızı (nadir)
  * 1: Düz eğimli: minimal değişiklik (tipik sağlıklı kalp)
  * 2: Downslopins: sağlıksız kalbin belirtileri
12. ca: florosopi ile renklendirilen ana damarların sayısı (0-3)
  * renkli damar, doktorun içinden geçen kanı görebileceği anlamına gelir
  * ne kadar çok kan hareketi olursa o kadar iyi (pıhtı olmaz)
13. thal: talyum stres sonucu
  * 1,3: normal
  * 6: Düzeltilen kusur: eskiden kusur vardı ama şimdi sorun yok
  * 7: geri döndürülebilir kusur: egzersiz sırasında uygun kan hareketi yok
14. target: hastalığın olup olmadığı (1=evet, 0=hayır) (tahmin edilen özellik)

# 3. Kütüphaneler ve Veri Setinin içe Aktarılması

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, train_test_split, cross_val_score, RandomizedSearchCV, GridSearchCV

from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score

from sklearn.preprocessing import StandardScaler

In [ ]:
df_ = pd.read_csv("/kaggle/input/heart-disease/heart.csv")
df = df_.copy()

In [ ]:
df.shape

Veri seti 303 satır 14 sütun içeriyor.

# Keşifsel veri analizi (EDA)

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df['target'].value_counts()

Veri setinde 165 hastada kalp hastalığı bulunuyor, 138 hastada ise kalp hastalığı bulunmuyor.

In [ ]:
df['target'].value_counts().plot(kind = 'bar', color = ['blue', 'black'])

In [ ]:
df.info()

Veri setinde eksik değerleri gözlemliyoruz ve tüm sütunlarda sayısal veri türleri bulunuyor.

In [ ]:
df.isna().sum()

In [ ]:
df.describe()

In [ ]:
df['sex'].value_counts()

Cinsiyet sütununda 1 erkekleri 0 ise kadınları temsil ettiğinden veri setinde 207 erkek ve 96 kadın olduğunu görebiliriz. Hedef sütunla karşılaştırma yapalım.

In [ ]:
pd.crosstab(df['target'],df['sex'])

Karşılaştırmaya bakıldığında kadınlarda kalp hastalığı oranı erkeklere göre daha yüksek.

In [ ]:
pd.crosstab(df['target'],df['sex']).plot(kind = 'bar', fig = (8,4), color = ['yellow', 'red'])
plt.title('Cinsiyete Göre Kalp Hastalığı Sıklığı')
plt.xlabel('0 = Hastalık, 1 = Hastalık Yok')
plt.ylabel('Miktar')
plt.legend(['kadın','erkek'])
plt.show()

In [ ]:
plt.scatter(df.age[df.target == 1],
           df.thalach[df.target == 1],
c = 'yellow')
plt.scatter(df.age[df.target == 0],
           df.thalach[df.target == 0],
           c = 'red')
plt.title('Maksimum Kalp Atış Hızı ve Yaş İlişkisi')
plt.xlabel('Age')
plt.ylabel('Maksimum Kalp Atış Hızı')
plt.legend(['hastalık', 'hastalık yok'])
plt.show()

In [ ]:
pd.crosstab(df.cp, df.target)

In [ ]:
pd.crosstab(df.cp, df.target).plot(kind = 'bar', figsize = (8,4), color = ['yellow', 'red'])
plt.title('Göğüs Tipinin Hastalıklarla İlişkisi')
plt.xlabel('Tip')
plt.ylabel('Miktar')
plt.legend(['Disease','No Disease'])
plt.show()

In [ ]:
import sys
print(sys.version)

In [ ]:
corr_matrix = df.corr()
fig ,ax = plt.subplots(figsize = (15,10))
ax = sns.heatmap(corr_matrix,
                 annot=True,
                 linewidths=0.5,
                 fmt='.2f',
                 cmap = 'YlGnBu')
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)

# Modelleme

In [ ]:
df.head()

In [ ]:
x = df.drop(columns=['target'], axis = 1)
y = df['target']

In [ ]:
x.head()

In [ ]:
y.head()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, stratify = y, random_state = 20)

Verileri eğitim ve test kümelerine böldük. Verilerin %20 'sini test seti olarak belirledik. Sınıflandırma problemi olduğu için verilen parametreyi y olarak katmanlaştırdık. Aynı sonuçları elde etmek için random_state parametresine 20 değerini verdik.

In [ ]:
len(y_train)

Model seçim sürecini değerlendirmek için bir işlem hattı oluşturabiliriz. Gridsearch ile farklı parametre değerlerine sahip 4 farklı makine öğrenmesi modeli deneyeceğiz. Çapraz doğrulama yöntemi olarak 5 bölmeli KFold (katmanlı) kullanacağız.

1. Logistic Regression
2. K-Nearest Neighbours Classifier
3. Random Forest Classifier
4. Boosting Classifier


In [ ]:
pipe = Pipeline([('scaler', StandardScaler()), ('Classifier', RandomForestClassifier())])
search_space = [
    {'Classifier': [LogisticRegression(solver='liblinear')],
     'Classifier__C': np.logspace(-4, 4, 100),
     'Classifier__penalty':['l1','l2']
    },
    {'Classifier': [KNeighborsClassifier()],
     'Classifier__n_neighbors':np.arange(1,10)
    },
    {'Classifier': [RandomForestClassifier(random_state=20)],
     'Classifier__n_estimators': [300,400,500],
     'Classifier__max_features': np.arange(1,10)
    },
    {'Classifier': [GradientBoostingClassifier(random_state=10)],
     'Classifier__n_estimators': [100, 500, 1000],
     'Classifier__learning_rate':[0.001,0.01,0.1]
    }
]
clf = GridSearchCV(pipe, search_space, cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=3), verbose=0, scoring='accuracy')
best_model = clf.fit(x_train, y_train)
best_model.best_estimator_.get_params()

Bu kod tüm süreci kapsar ve en iyi modelin yapılandırmasını kontrol etmeye yarar. GridSearch sonuçlarına göre en iyi modelin Lojistik Regresyon olduğu görülmektedir. Artık modeli, verilen C=0,1176811952434999 parametre değeriyle y değerlerini tahmin etmek için deneyebiliriz.

In [ ]:
scaler = StandardScaler()
scaler.fit(x_train)
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [ ]:
model = LogisticRegression(C=0.1176811952434999, penalty='l1', solver='liblinear')
model.fit(x_train_scaled, y_train)

In [ ]:
y_preds = model.predict(x_test_scaled)

In [ ]:
fpr, tpr, _ = roc_curve(y_test, y_preds)
auc = roc_auc_score(y_test, y_preds)
plt.plot(fpr, tpr, label = 'data 1, auc=' + str(auc))
plt.legend(loc = 4)
plt.show()

In [ ]:
print(confusion_matrix(y_test, y_preds))

In [ ]:
print(classification_report(y_test, y_preds))

Model, pozitif sınıfı (1) oldukça iyi tahmin ederken, negatif sınıfı (0) için bazı yanlış tahminler yapmıştır. Genel olarak, modelin performansı iyi görünmektedir.

In [ ]:
model.coef_

model.coef_, modelin karar verme sürecini anlamak ve hangi özelliklerin daha önemli olduğunu değerlendirmek için kullanılır. 0.04632227, -0.3587711, -0.04557311 gibi değerler, bu özelliklerin pozitif sınıfı azaltma eğiliminde olduğunu gösterir. 0.48575883, 0.04714858 gibi değerler ise bu özelliklerin pozitif sınıfı artırma eğiliminde olduğunu gösterir. 0. değerleri, ilgili özelliklerin modelin tahminlerine hiçbir katkı sağlamadığını ifade eder. Yani, bu özellikler model tarafından dikkate alınmamıştır.

In [ ]:
feature_dict = dict(zip(df.columns, list(model.coef_[0])))
feature_dict

Hangi özelliklerin pozitif veya negatif yönde daha etkili olduğu görülmektedir.

In [ ]:
feature_df = pd.DataFrame(feature_dict, index=[0])
feature_df.T.plot.bar(title="Katsayılar", legend=False);

Bu görselleştirme, hangi özelliklerin model için daha önemli olduğunu hızlıca anlamanızı sağlar.